# Sitemap

`SitemapLoader` 扩展自 `WebBaseLoader`，它从给定的 URL 加载站点地图，然后抓取并加载站点地图中的所有页面，将每个页面作为一个 Document 返回。

抓取过程是并发进行的。并发请求有合理的限制，默认为每秒 2 次。如果你不担心做个“好公民”，或者你控制被抓取的服务器，或者不关心负载，可以增加这个限制。请注意，虽然这会加快抓取过程，但可能会导致服务器阻止你。请小心！

## 概览
### 集成详情

| 类 | 包 | 本地 | 可序列化 | [JS 支持](https://js.langchain.com/docs/integrations/document_loaders/web_loaders/sitemap/)|
| :--- | :--- | :---: | :---: |  :---: |
| [SiteMapLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.sitemap.SitemapLoader.html#langchain_community.document_loaders.sitemap.SitemapLoader) | [langchain_community](https://python.langchain.com/api_reference/community/index.html) | ✅ | ❌ | ✅ |
### 加载器功能
| 源 | 文档延迟加载 | 原生异步支持
| :---: | :---: | :---: |
| SiteMapLoader | ✅ | ❌ |

## 设置

要访问 SiteMap 文档加载器，你需要安装 `langchain-community` 集成包。

### 凭证

运行此程序不需要任何凭证。

为了启用模型调用的自动化追踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

安装 **langchain_community**。

In [ ]:
%pip install -qU langchain-community

### 修复笔记本 asyncio 错误

In [4]:
import nest_asyncio

nest_asyncio.apply()

## 初始化

现在我们可以实例化模型对象并加载文档了：

In [ ]:
from langchain_community.document_loaders.sitemap import SitemapLoader

In [ ]:
sitemap_loader = SitemapLoader(web_path="https://api.python.langchain.com/sitemap.xml")

## 加载

In [5]:
docs = sitemap_loader.load()
docs[0]

Fetching pages: 100%|##########| 28/28 [00:04<00:00,  6.83it/s]


Document(metadata={'source': 'https://api.python.langchain.com/en/stable/', 'loc': 'https://api.python.langchain.com/en/stable/', 'lastmod': '2024-05-15T00:29:42.163001+00:00', 'changefreq': 'weekly', 'priority': '1'}, page_content='\n\n\n\n\n\n\n\n\n\nLangChain Python API Reference Documentation.\n\n\nYou will be automatically redirected to the new location of this page.\n\n')

In [6]:
print(docs[0].metadata)

{'source': 'https://api.python.langchain.com/en/stable/', 'loc': 'https://api.python.langchain.com/en/stable/', 'lastmod': '2024-05-15T00:29:42.163001+00:00', 'changefreq': 'weekly', 'priority': '1'}


您可以更改 **`requests_per_second`**  参数来提高最大并发请求数，并使用 **`requests_kwargs`** 在发送请求时传递 kwargs。

In [ ]:
sitemap_loader.requests_per_second = 2
# Optional: avoid `[SSL: CERTIFICATE_VERIFY_FAILED]` issue
sitemap_loader.requests_kwargs = {"verify": False}

## 延迟加载

您也可以延迟加载页面以最小化内存占用。

In [7]:
page = []
for doc in sitemap_loader.lazy_load():
    page.append(doc)
    if len(page) >= 10:
        # do some paged operation, e.g.
        # index.upsert(page)

        page = []

Fetching pages: 100%|##########| 28/28 [00:01<00:00, 19.06it/s]


## 过滤站点地图 URL

站点地图文件可能非常庞大，包含数千个 URL。通常情况下，您不需要它们中的每一个。您可以将字符串或正则表达式模式的列表传递给 `filter_urls` 参数来过滤 URL。只有匹配其中一个模式的 URL 才会被加载。

In [ ]:
loader = SitemapLoader(
    web_path="https://api.python.langchain.com/sitemap.xml",
    filter_urls=["https://api.python.langchain.com/en/latest"],
)
documents = loader.load()

In [8]:
documents[0]

Document(page_content='\n\n\n\n\n\n\n\n\n\nLangChain Python API Reference Documentation.\n\n\nYou will be automatically redirected to the new location of this page.\n\n', metadata={'source': 'https://api.python.langchain.com/en/latest/', 'loc': 'https://api.python.langchain.com/en/latest/', 'lastmod': '2024-02-12T05:26:10.971077+00:00', 'changefreq': 'daily', 'priority': '0.9'})

## 添加自定义抓取规则

`SitemapLoader` 在抓取过程中使用 `beautifulsoup4`，默认情况下会抓取页面上的所有元素。`SitemapLoader` 构造函数接受一个自定义的抓取函数。此功能有助于根据您的特定需求定制抓取过程；例如，您可能希望避免抓取标题或导航元素。

下面的示例展示了如何开发和使用自定义函数来避免导航和标题元素。

导入 `beautifulsoup4` 库并定义自定义函数。

In [ ]:
pip install beautifulsoup4

In [10]:
from bs4 import BeautifulSoup


def remove_nav_and_header_elements(content: BeautifulSoup) -> str:
    # Find all 'nav' and 'header' elements in the BeautifulSoup object
    nav_elements = content.find_all("nav")
    header_elements = content.find_all("header")

    # Remove each 'nav' and 'header' element from the BeautifulSoup object
    for element in nav_elements + header_elements:
        element.decompose()

    return str(content.get_text())

将你的自定义函数添加到 `SitemapLoader` 对象。

In [11]:
loader = SitemapLoader(
    "https://api.python.langchain.com/sitemap.xml",
    filter_urls=["https://api.python.langchain.com/en/latest/"],
    parsing_function=remove_nav_and_header_elements,
)

##  локальная карта сайта

Локальный загрузчик карты сайта также может использоваться для загрузки локальных файлов.

In [ ]:
sitemap_loader = SitemapLoader(web_path="example_data/sitemap.xml", is_local=True)

docs = sitemap_loader.load()

## API 参考

有关 SiteMapLoader 所有功能和配置的详细文档，请访问 API 参考：https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.sitemap.SitemapLoader.html#langchain_community.document_loaders.sitemap.SitemapLoader